In [18]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
credits=pd.read_csv('credits.csv')
movies=pd.read_csv('movies.csv')

In [20]:
print(movies)

         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [21]:
credits.head(1)['crew'].values

array(['[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cam

In [22]:
movies=movies.merge(credits,on='title')

In [23]:
#list of Movies 
#genres
#id 
#keywords
#title
#overview
#cost 
#crew
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]


In [24]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [25]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [26]:
movies.dropna(inplace=True)

In [27]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [28]:
movies.duplicated().sum()

np.int64(0)

In [29]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [30]:
movies['genres']=movies['genres'].apply(convert)

In [31]:
movies['keywords']=movies['keywords'].apply(convert)

In [32]:
def convertCast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
          L.append(i['name'])
          counter+=1
        else:
            break
    return L

In [33]:
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [34]:
movies['cast'].apply(convertCast)

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4806, dtype: object

In [35]:
def fetchDirector(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            L.append(i['name'])
            break
    return L

In [36]:
movies['crew']=movies['crew'].apply(fetchDirector)

In [37]:
movies.head(10)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...",[Andrew Stanton]
5,559,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,"[Fantasy, Action, Adventure]","[dual identity, amnesia, sandstorm, love of on...","[{""cast_id"": 30, ""character"": ""Peter Parker / ...",[Sam Raimi]
6,38757,Tangled,When the kingdom's most wanted-and most charmi...,"[Animation, Family]","[hostage, magic, horse, fairy tale, musical, p...","[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...",[Byron Howard]
7,99861,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,"[Action, Adventure, Science Fiction]","[marvel comic, sequel, superhero, based on com...","[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...",[Joss Whedon]
8,767,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...","[Adventure, Fantasy, Family]","[witch, magic, broom, school of witchcraft, wi...","[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...",[David Yates]
9,209112,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,"[Action, Adventure, Fantasy]","[dc comics, vigilante, superhero, based on com...","[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...",[Zack Snyder]


In [38]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [39]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])



In [40]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [41]:
print(movies)

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                               overview  \
0     [In, the, 22nd, century,, a, paraplegic, Marin...   
1     [Captain, Barbossa,, long, believed, to, be, d...   
2     [A, cryptic, message, from, Bond’s, past, send...   
3     [Following, the, death, of, District,

In [42]:
df=movies[['movie_id','title','tags']]

In [43]:
df['tags']=df['tags'].apply(lambda x:" ".join(x))

C:\Users\hp\AppData\Local\Temp\ipykernel_4348\2493417758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:" ".join(x))


In [44]:
df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d Action Adventure Fantasy ScienceFiction JamesCameron'

In [45]:
df['tags']=df['tags'].apply(lambda x:x.lower())

C:\Users\hp\AppData\Local\Temp\ipykernel_4348\2034404066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:x.lower())


In [46]:
vectors=cv.fit_transform(df['tags']).toarray()

In [47]:
len(cv.get_feature_names_out())

5000

In [48]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [49]:
df['tags']=df['tags'].apply(stem)

C:\Users\hp\AppData\Local\Temp\ipykernel_4348\4230616273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(stem)


In [50]:
similarity=cosine_similarity(vectors)

In [51]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

[(0, np.float64(1.0000000000000002)),
 (1192, np.float64(0.4745789978762496)),
 (61, np.float64(0.47434164902525694)),
 (1440, np.float64(0.4213504858001923)),
 (2997, np.float64(0.40946151242666284)),
 (322, np.float64(0.4082482904638631)),
 (260, np.float64(0.4057513356003446)),
 (495, np.float64(0.40204220056417306)),
 (46, np.float64(0.40089186286863665)),
 (466, np.float64(0.3888568855239095)),
 (931, np.float64(0.38306543884143696)),
 (208, np.float64(0.3830161731427133)),
 (539, np.float64(0.37500000000000006)),
 (83, np.float64(0.36661778755338326)),
 (507, np.float64(0.36661778755338326)),
 (972, np.float64(0.365781760475145)),
 (1654, np.float64(0.3651483716701108)),
 (1202, np.float64(0.3638034375544995)),
 (973, np.float64(0.3617840255724682)),
 (232, np.float64(0.35634832254989923)),
 (3728, np.float64(0.35355339059327384)),
 (10, np.float64(0.3521803625302496)),
 (2492, np.float64(0.3517632353407243)),
 (7, np.float64(0.3503245248726854)),
 (1392, np.float64(0.35007002100

In [52]:
def recommender(movie):
    movieIndex=df[df['title']==movie].index[0]
    distances=similarity[movieIndex]
    movies_list=sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(df.iloc[i[0]].title)
    

In [55]:
recommender('Avatar')

Small Soldiers
Jupiter Ascending
Krull
Mad Max Beyond Thunderdome
The Fifth Element
